In [1]:
import os
from transformers import pipeline
import ffmpeg as ff


# 전체 오디오 추출
def extract_audio(video_path, audio_path):
    ff.input(video_path).output(audio_path).run()


# Whisper 모델로 오디오 처리
def transcribe_audio(audio_path):
    transcriber = pipeline(
        "automatic-speech-recognition", model="openai/whisper-medium"
    )
    result = transcriber(audio_path)
    return result["text"]


# 텍스트를 파일로 저장
def save_transcript_to_file(transcript, file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(transcript)


# 폴더 생성
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)


# 폴더 내 모든 오디오 파일 처리
def process_audio_files_in_folder(folder_path):
    text_folder_path = os.path.join(folder_path, "text")
    create_folder_if_not_exists(text_folder_path)

    for filename in os.listdir(folder_path):
        if filename.endswith(".mp3"):  # MP3 파일 포맷으로 변경
            audio_path = os.path.join(folder_path, filename)
            text_file_path = os.path.join(
                text_folder_path, f"{os.path.splitext(filename)[0]}_transcript.txt"
            )

            # 추출된 오디오를 Whisper로 처리
            transcript = transcribe_audio(audio_path)

            # 텍스트 파일로 저장
            save_transcript_to_file(transcript, text_file_path)

            print(f"Transcript for {filename} saved to {text_file_path}")


# 예시 사용
folder_path = "./lecture_video/audio/4_2"  # 각자 디렉토리에 맞게 수정해야 함
process_audio_files_in_folder(folder_path)

c:\Users\user\anaconda3\envs\4th_math\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import openai
import os


# 로컬 txt 파일 읽기 함수
def read_text_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()
    return content


# 텍스트 파일로 저장하기
def save_combined_content(content, base_file_name, folder_path):
    summary_folder = os.path.join(folder_path, "summary")
    with open(
        os.path.join(summary_folder, f"{base_file_name}_processed.txt"),
        "w",
        encoding="utf-8",
    ) as file:
        file.write(content)


# ChatCompletion 호출 및 응답 처리
def process_text_with_openai(file_content):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",  # gpt-4o 로 해도 됨
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a helpful assistant that specializes in correcting text "
                    "related to South Korean elementary 4th grade math class. "
                    "Please correct any awkward or incorrect sentences in the provided material. "
                    "Additionally, please remove any unnecessary or redundant information from the text. "
                    "Finally, separate the content into '수업내용', '요약', and '키워드' sections."
                ),
            },
            {
                "role": "user",
                "content": (
                    f"한국 초등학교 4학년 수학 수업 내용을 기반으로 교정해 줘"
                    f"그리고 불필요하거나 중복된 내용을 제거하고, 수업 내용, 요약, 키워드로 나누어 줘"
                    f"수업 내용 최대 글자수 600자로 설정해주고, 각 문장은 줄바꿈 없이 붙여줘"
                    f"요약만 항목으로 나타내주고, 각 항목은 - 로 시작해줘. 키워드는 항목으로 나타내지 않고 콤마(,)로 연결해줘"
                    f"출력된 키워드에 숫자가 포함된다면 그 키워드는 그냥 제거해줘"
                    f"수업 내용은 '~합니다','습니다' 등의 격식체로 해줘"
                    f"수업 내용:, 요약:, 키워드: 이런식으로 제목적어줘"
                    f"요약은 5개 항목으로 나타내주고 온점(.) 으로 끝나지 않게 해줘"
                    f"요약의 각 항목은 줄바꿈해줘"
                    f"요약 말투는 격식체나 평서체가 아니게 해줘. 예를 들면 '~함', '~임', '~사용'"
                    f"내용을 가능한 한 유지하고 첫인사와 마지막 인사는 제거해줘: {file_content}"
                ),
            },
        ],
        temperature=0,  # 랜덤성을 줄이고 결정론적 출력을 위한 설정
        top_p=1  # 모든 토큰을 고려하여 더 결정적인 결과를 얻음
    )
    return response["choices"][0]["message"]["content"]


# 폴더 내 모든 텍스트 파일 처리
def process_text_files_in_folder(folder_path):
    text_files = [f for f in os.listdir(folder_path) if f.endswith(".txt")]

    for text_file in text_files:
        file_path = os.path.join(folder_path, text_file)
        file_content = read_text_file(file_path)

        # OpenAI API 호출하여 내용 처리
        processed_text = process_text_with_openai(file_content)

        # 결과를 하나의 파일로 저장
        base_file_name = os.path.splitext(text_file)[
            0
        ]  # 원본 파일 이름에서 확장자 제외
        save_combined_content(processed_text, base_file_name, folder_path)
        print(f"Processed and saved combined content for {text_file}")


# 예시 사용
folder_path = "./lecture_video/audio/4_2/text"  # 텍스트 파일이 저장된 폴더 경로
process_text_files_in_folder(folder_path)